In [1]:
import os
import pandas as pd
import json
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI 

In [2]:
# load the env variables defined in local .env file
from dotenv import load_dotenv
load_dotenv()

True

In [85]:
# load the env variable as defined in .env file
My_KEY=os.getenv("OPENAI_API_KEY2") # OPENAI_API_KEY


In [86]:
# temperature value could be 0-2. 0 means non creative straightforward answer. 2 means completely creative answer
llm=ChatOpenAI(openai_api_key=My_KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [87]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [89]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [91]:
Quiz_tmpl="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [92]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=Quiz_tmpl
    )

In [93]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [94]:
Quiz_Review_Tmpl="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [95]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=Quiz_Review_Tmpl)

In [96]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [98]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [99]:
file_path=r"C:\genaipjts\mcqgen\data.txt"

In [33]:
file_path

'C:\\genaipjts\\mcqgen\\data.txt'

In [100]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [35]:
print(TEXT)

The Telecom Regulatory Authority of India released a consultation paper on “Inputs for Formulation of National Broadcasting Policy, 2024”.[28]  In September 2023, TRAI had released a pre-consultation paper seeking inputs for formulating a National Broadcasting Policy.[29]  The Ministry of Information and Broadcasting had also released a draft Broadcasting Services (Regulation) Bill, 2023 for comments in November 2023.[30]  TRAI has sought views on key issues such as:

Growth of the broadcasting sector:  As per TRAI, there are approximately 100 million homes without television.  It has invited suggestions on strategies for making TV services more affordable.  The consultation paper also observes a heavy reliance on imported equipment in the sector.  This includes imported devices and components for assembly. Hence, it has sought inputs on promoting R&D and domestic manufacturing. 

India as a global content hub:  The paper cites various methods used by other countries to promote local c

In [101]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [102]:
NUMBER=2 
SUBJECT="public policy"
TONE="simple"

In [103]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The Telecom Regulatory Authority of India released a consultation paper on “Inputs for Formulation of National Broadcasting Policy, 2024”.[28]  In September 2023, TRAI had released a pre-consultation paper seeking inputs for formulating a National Broadcasting Policy.[29]  The Ministry of Information and Broadcasting had also released a draft Broadcasting Services (Regulation) Bill, 2023 for comments in November 2023.[30]  TRAI has sought views on key issues such as:

Growth of the broadcasting sector:  As per TRAI, there are approximately 100 million homes without television.  It has invited suggestions on strategies for making TV services more affordable.  The consultation paper also observes a heavy reliance on imported equipment in the sector.  This includes imported devices and components for assembly. Hence, it has sought inputs on promoting R&D and domestic manufacturing. 

In [104]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1222
Prompt Tokens:978
Completion Tokens:244
Total Cost:0.001955


In [106]:
response

{'text': 'The Telecom Regulatory Authority of India released a consultation paper on “Inputs for Formulation of National Broadcasting Policy, 2024”.[28]  In September 2023, TRAI had released a pre-consultation paper seeking inputs for formulating a National Broadcasting Policy.[29]  The Ministry of Information and Broadcasting had also released a draft Broadcasting Services (Regulation) Bill, 2023 for comments in November 2023.[30]  TRAI has sought views on key issues such as:\n\nGrowth of the broadcasting sector:  As per TRAI, there are approximately 100 million homes without television.  It has invited suggestions on strategies for making TV services more affordable.  The consultation paper also observes a heavy reliance on imported equipment in the sector.  This includes imported devices and components for assembly. Hence, it has sought inputs on promoting R&D and domestic manufacturing. \n\nIndia as a global content hub:  The paper cites various methods used by other countries to p

In [108]:
print(response.get("quiz"))


{
    "1": {
        "mcq": "What is one of the key issues discussed in the consultation paper on National Broadcasting Policy, 2024?",
        "options": {
            "a": "Promoting foreign equipment imports",
            "b": "Strategies for making TV services more affordable",
            "c": "Reducing local content quotas",
            "d": "Encouraging piracy in the film industry"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "What is one of the measures suggested to combat piracy and enhance content security?",
        "options": {
            "a": "Increasing import of pirated content",
            "b": "Promoting unauthorised distribution of content",
            "c": "Taking additional measures to combat piracy",
            "d": "Ignoring the issue of content security"
        },
        "correct": "c"
    }
}


In [109]:
quiz=json.loads(response.get("quiz"))

In [110]:
quiz

{'1': {'mcq': 'What is one of the key issues discussed in the consultation paper on National Broadcasting Policy, 2024?',
  'options': {'a': 'Promoting foreign equipment imports',
   'b': 'Strategies for making TV services more affordable',
   'c': 'Reducing local content quotas',
   'd': 'Encouraging piracy in the film industry'},
  'correct': 'b'},
 '2': {'mcq': 'What is one of the measures suggested to combat piracy and enhance content security?',
  'options': {'a': 'Increasing import of pirated content',
   'b': 'Promoting unauthorised distribution of content',
   'c': 'Taking additional measures to combat piracy',
   'd': 'Ignoring the issue of content security'},
  'correct': 'c'}}

In [112]:
print(quiz.items())

dict_items([('1', {'mcq': 'What is one of the key issues discussed in the consultation paper on National Broadcasting Policy, 2024?', 'options': {'a': 'Promoting foreign equipment imports', 'b': 'Strategies for making TV services more affordable', 'c': 'Reducing local content quotas', 'd': 'Encouraging piracy in the film industry'}, 'correct': 'b'}), ('2', {'mcq': 'What is one of the measures suggested to combat piracy and enhance content security?', 'options': {'a': 'Increasing import of pirated content', 'b': 'Promoting unauthorised distribution of content', 'c': 'Taking additional measures to combat piracy', 'd': 'Ignoring the issue of content security'}, 'correct': 'c'})])


In [115]:
" <> ".join(["deepak","kumar","nema"])

'deepak <> kumar <> nema'

In [117]:
quiz_table_data = []
for key, value in quiz.items():
    print(f"key - {key}")
    print(f"value - {value}")
   
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]

    print(f"mcq - {mcq}")
    print(f"correct - {correct}")
    print(f"options - {options}")  

    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

key - 1
value - {'mcq': 'What is one of the key issues discussed in the consultation paper on National Broadcasting Policy, 2024?', 'options': {'a': 'Promoting foreign equipment imports', 'b': 'Strategies for making TV services more affordable', 'c': 'Reducing local content quotas', 'd': 'Encouraging piracy in the film industry'}, 'correct': 'b'}
mcq - What is one of the key issues discussed in the consultation paper on National Broadcasting Policy, 2024?
correct - b
options - a: Promoting foreign equipment imports | b: Strategies for making TV services more affordable | c: Reducing local content quotas | d: Encouraging piracy in the film industry
key - 2
value - {'mcq': 'What is one of the measures suggested to combat piracy and enhance content security?', 'options': {'a': 'Increasing import of pirated content', 'b': 'Promoting unauthorised distribution of content', 'c': 'Taking additional measures to combat piracy', 'd': 'Ignoring the issue of content security'}, 'correct': 'c'}
mcq 

In [119]:
quiz_table_data

[{'MCQ': 'What is one of the key issues discussed in the consultation paper on National Broadcasting Policy, 2024?',
  'Choices': 'a: Promoting foreign equipment imports | b: Strategies for making TV services more affordable | c: Reducing local content quotas | d: Encouraging piracy in the film industry',
  'Correct': 'b'},
 {'MCQ': 'What is one of the measures suggested to combat piracy and enhance content security?',
  'Choices': 'a: Increasing import of pirated content | b: Promoting unauthorised distribution of content | c: Taking additional measures to combat piracy | d: Ignoring the issue of content security',
  'Correct': 'c'}]

In [121]:
quiz_df = pd.DataFrame(quiz_table_data)

quiz_df

,MCQ,Choices,Correct
0,What is one of the key issues discussed in the...,a: Promoting foreign equipment imports | b: St...,b
1,What is one of the measures suggested to comba...,a: Increasing import of pirated content | b: P...,c


In [122]:
quiz_df.to_csv("mcq_quiz.csv",index=False)

In [123]:
# for experimentation with index true

quiz_df.to_csv("mcq_quiz2.csv",index=True)

In [124]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_11_2024_12_48_58'

In [1]:
import os
print(os.getcwd())

c:\genaipjts\mcqgen\experiment
